In [1]:
from sklearn.datasets import make_regression
from matplotlib import pyplot
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# generate regression dataset
X, y = make_regression(n_samples=1000, n_features=4, noise=10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

pd.DataFrame(data = np.concatenate((X_train, np.array([y_train]).T), axis=1)).to_csv("train.csv", index=None)
pd.DataFrame(data = np.concatenate((X_test, np.array([y_test]).T), axis=1)).to_csv("test.csv", index=None)

In [2]:
reg = LinearRegression().fit(X_train, y_train)

In [3]:
mean_absolute_error(y_test, reg.predict(X_test))

7.748804690163286